## 0. import some important package

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## 1. load the data 

Use the data from "[_Microsoft Anonymous Web Data_](http://kdd.ics.uci.edu/databases/msweb/msweb.html)".  
Notice: from the describtion }(http://kdd.ics.uci.edu/databases/msweb/msweb.html), the data in file mainly have 2 parts, "attribute" and "user".


In [14]:
attribute_df = pd.read_csv("anonymous-msweb.data",header=None,skiprows=range(0,7),nrows=293, names=['marks', 'attribute ID','ignored','title','url'])
attribute_df.sample(5)

,marks,attribute ID,ignored,title,url
245,A,1040,1,MS Office Info,/office
258,A,1101,1,Microsoft OLE DB,/oledb
56,A,1078,1,NT Server Support,/ntserversupport
87,A,1143,1,Site Builder Workshop,/workshoop
152,A,1273,1,mdn,/mdn


In [9]:
user_df = pd.read_csv("anonymous-msweb.data",header=None,skiprows=range(0,301), names=['marks', 'ID','ignored'])
user_df.head(5)

,marks,ID,ignored
0,C,10001,10001
1,V,1000,1
2,V,1001,1
3,V,1002,1
4,C,10002,10002


In [15]:
url_save = attribute_df['url']
attribute_df.drop(['marks','ignored','url'],axis=1,inplace=True)

,attribute ID,title
0,1287,International AutoRoute
1,1288,library
2,1289,Master Chef Product Information
3,1297,Central America
4,1215,For Developers Only Info
5,1279,Multimedia Golf
6,1239,Microsoft Consulting
7,1282,home
8,1251,Reference Support
9,1121,Microsoft Magazine


In [16]:
attribute_df.sample(5)

,attribute ID,title
47,1074,Windows NT Workstation
224,1246,Developer Media Games
38,1284,partner
22,1005,Norway
163,1186,Job Listings for Pre-Grads


In [18]:
user = []
record = []
temp_sequence = None
for index, row in user_df.iterrows():
    if row['marks']=='C':
        if temp_sequence!= None:
            record.append(temp_sequence)
        user.append(row['ID'])
        temp_sequence=[]
    else:
        temp_sequence.append(row['ID'])
record.append(temp_sequence)
user_df = pd.DataFrame({"user":user,"visit":record})


,user,visit
32706,42707,"[1008, 1030, 1009, 1058, 1004, 1018]"
32707,42708,"[1008, 1027, 1123, 1038, 1026, 1041]"
32708,42709,"[1001, 1003]"
32709,42710,"[1035, 1001, 1018]"
32710,42711,[1008]


In [19]:
user_df.sample(5)

,user,visit
11866,21867,[1004]
13399,23400,"[1045, 1008, 1025, 1017, 1034]"
592,10593,[1004]
27089,37090,[1004]
9562,19563,"[1008, 1017, 1134, 1034, 1004]"
